# 1.DATA

In [1]:
import numpy as np
from datetime import datetime as dt
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
with open ('sx-stackoverflow-a2q.txt') as dataset_a2q:
    a2q=dataset_a2q.readlines()

with open ('sx-stackoverflow-c2a.txt') as dataset_c2a:
    c2a= dataset_c2a.readlines()
    
with open ('sx-stackoverflow-c2q.txt') as dataset_c2q:
    c2q= dataset_c2q.readlines()

### Preparing the data

Note that we're changing the values of the read data instead of creating new variables to store the corrected version, in order to save space.

In [3]:
n=10000 #we'll try first with n=100 rows from each one of the 3 datasets
for i in range(0,n):
    a2q[i]=a2q[i].split()
    a2q[i][0]=int(a2q[i][0])
    a2q[i][1]=int(a2q[i][1])
    a2q[i][2]=dt.utcfromtimestamp(int(a2q[i][2])).strftime('%Y-%m-%d')
    
for i in range(0,n):
    c2a[i]=c2a[i].split()
    c2a[i][0]=int(c2a[i][0])
    c2a[i][1]=int(c2a[i][1])
    c2a[i][2]=dt.utcfromtimestamp(int(c2a[i][2])).strftime('%Y-%m-%d')
    
for i in range(0,n):
    c2q[i]=c2q[i].split()
    c2q[i][0]=int(c2q[i][0])
    c2q[i][1]=int(c2q[i][1])
    c2q[i][2]=dt.utcfromtimestamp(int(c2q[i][2])).strftime('%Y-%m-%d')

## Faster version, but gives memory error

In [4]:
'''
with open ('sx-stackoverflow-a2q.txt') as dataset_a2q:
    a2q = [int(x) for x in dataset_a2q.read().split()] #while reading lines, split and transform into int
    #len(lst)=3* real number of lines, because after this step each value is stored alone: no nested lists
'''

"\nwith open ('sx-stackoverflow-a2q.txt') as dataset_a2q:\n    a2q = [int(x) for x in dataset_a2q.read().split()] #while reading lines, split and transform into int\n    #len(lst)=3* real number of lines, because after this step each value is stored alone: no nested lists\n"

In [5]:
'''
m=int(len(a2q)/3)
for i in range(2,m+1,3):
    a2q[i]=dt.utcfromtimestamp(a2q[i]).strftime('%Y-%m-%d')
    '''

"\nm=int(len(a2q)/3)\nfor i in range(2,m+1,3):\n    a2q[i]=dt.utcfromtimestamp(a2q[i]).strftime('%Y-%m-%d')\n    "

In [6]:
'''
with open ('sx-stackoverflow-c2a.txt') as dataset_c2a:
    c2a = [int(x) for x in dataset_c2a.read().split()]
    '''

"\nwith open ('sx-stackoverflow-c2a.txt') as dataset_c2a:\n    c2a = [int(x) for x in dataset_c2a.read().split()]\n    "

In [7]:
'''
m=int(len(c2a)/3)
for i in range(2,m+1,3):
    c2a[i]=dt.utcfromtimestamp(c2a[i]).strftime('%Y-%m-%d')
'''

"\nm=int(len(c2a)/3)\nfor i in range(2,m+1,3):\n    c2a[i]=dt.utcfromtimestamp(c2a[i]).strftime('%Y-%m-%d')\n"

In [8]:
'''
with open ('sx-stackoverflow-c2q.txt') as dataset_c2q:
    c2q = [int(x) for x in dataset_c2q.read().split()]
'''

"\nwith open ('sx-stackoverflow-c2q.txt') as dataset_c2q:\n    c2q = [int(x) for x in dataset_c2q.read().split()]\n"

In [9]:
'''
m=int(len(c2q)/3)
for i in range(2,m+1,3):
    c2q[i]=dt.utcfromtimestamp(c2q[i]).strftime('%Y-%m-%d')
    '''

"\nm=int(len(c2q)/3)\nfor i in range(2,m+1,3):\n    c2q[i]=dt.utcfromtimestamp(c2q[i]).strftime('%Y-%m-%d')\n    "

### Remove answer from user to his own question, i.e. remove loops from corresponding graph

In [10]:
a2q_n=a2q[0:n]
c2a_n=c2a[0:n]
c2q_n=c2q[0:n]

In [11]:
w=0
while w<len(a2q_n):
    if a2q_n[w][0]==a2q_n[w][1]:
        del a2q_n[w]
    else:
        w+=1
naq=len(a2q_n)

### Should do the same for the other 2

In [12]:
x=0
while x<len(c2a_n):
    if c2a_n[x][0]==c2a_n[x][1]:
        del c2a_n[x]
    else:
        x+=1
nca=len(c2a_n)

In [13]:
len(c2a_n)

8319

In [14]:
y=0
while y<len(c2q_n):
    if c2q_n[y][0]==c2q_n[y][1]:
        del c2q_n[y]
    else:
        y+=1
ncq=len(c2q_n)

## NOW CREATE WEIGHTED GRAPH ??

### We'll use a nested dictionary

In [15]:
def dateInRange(date, initTime, endTime): #return true if the given date its in the range [initTime, endTime], return false otherwise
    if(initTime == '-' or endTime == '-'):
        return True
    if initTime <= date <= endTime:
        return True
    else:
        return False
    
def createGraph(initTime, endTime, data):
    graph = {}
    for x in range(len(data)):
        if dateInRange(data[x][2], initTime, endTime):
            if data[x][0] in graph: #we already have the node of that user in the graph
                if data[x][1] in graph[data[x][0]]: #we already have one interaction of user data[x][0] with user data[x][1], so we add 1 to the number of interactions
                    graph[data[x][0]][data[x][1]]['weight'] += 1
                else:
                    graph[data[x][0]][data[x][1]] = {'weight': 1} #we don't have an interacion between that 2 users, so we initialize the interaction with a 1 (because till the moment we only have 1 interaction).
            else: #we don't have the node of the user in the graph
                graph[data[x][0]] = {data[x][1] : {'weight': 1}}
    return graph

In [16]:
#a2q_n

In [17]:
'''
auxData = [['0','1','2008-02-11'], ['0', '2', '2008-02-11'], ['0', '1', '2008-02-11'], ['1','0', '2008-04-11']]
dict_a2q_n = createGraph('2008-02-11', '2008-02-11', auxData)
dict_a2q_n
'''

"\nauxData = [['0','1','2008-02-11'], ['0', '2', '2008-02-11'], ['0', '1', '2008-02-11'], ['1','0', '2008-04-11']]\ndict_a2q_n = createGraph('2008-02-11', '2008-02-11', auxData)\ndict_a2q_n\n"

In [18]:
dict_a2q_n = createGraph('-', '-', a2q_n)
dict_c2a_n = createGraph('-', '-', c2a_n)
dict_c2q_n = createGraph('-', '-', c2q_n)


In [19]:
#dict_a2q_n

In [20]:
'''
dict_a2q_n={}
for i in range (0,naq):
    dict_a2q_n[a2q_n[i][0]]={'a2vs_question':a2q_n[0][1],'time_a2q':a2q_n[0][2],'weight':0}
    '''

"\ndict_a2q_n={}\nfor i in range (0,naq):\n    dict_a2q_n[a2q_n[i][0]]={'a2vs_question':a2q_n[0][1],'time_a2q':a2q_n[0][2],'weight':0}\n    "

In [21]:
'''
dict_c2a_n={}
for i in range (0,nca):
    dict_c2a_n[c2a_n[i][0]]={'c2vs_answer':c2a_n[0][1],'time_c2a':c2a_n[0][2],'weight':0}
    '''

"\ndict_c2a_n={}\nfor i in range (0,nca):\n    dict_c2a_n[c2a_n[i][0]]={'c2vs_answer':c2a_n[0][1],'time_c2a':c2a_n[0][2],'weight':0}\n    "

In [22]:
'''
dict_c2q_n={}
for i in range (0,ncq):
    dict_c2q_n[c2q_n[i][0]]={'c2vs_question':c2q_n[0][1],'time_c2q':c2q_n[0][2],'weight':0}
    '''

"\ndict_c2q_n={}\nfor i in range (0,ncq):\n    dict_c2q_n[c2q_n[i][0]]={'c2vs_question':c2q_n[0][1],'time_c2q':c2q_n[0][2],'weight':0}\n    "

#### For the merged dataset: -----> to work on, according to how we decide to merge

In [62]:
def mergeTwoGraphs(g1, g2): #merge g1 and g2 into one graph. g1 and g2 must exist.
    #  print("-----------")
    #  print("g1: ", g1)
    #  print("g2: ", g2)
    #  print("-----------")
    for x in g2:
        #print("x: ", x)
        if x in g1: #if in g1 we have the user x, we have to add all his interactions 1 by 1. If the interaction already exists, we just add 1 to the weight.
            for y in g2[x]:
                #print("y: ", y)
                if y in g1[x]:
                    g1[x][y]['weight'] += g2[x][y]['weight']
                else:
                    g1[x][y] = {'weight': 1}
        else: #if in g1 we don't have the user x, we add all his interactions
            #print(x)
            g1[x] = g2[x]
    return g1
    
def mergeGraphs(graphs): #merge the graphs into one graph. The parameter graphs it's and array of all the graphs to merge [g1, g2, ...]
        mergedGraph = {}
        if len(graphs) > 0:
            mergedGraph = graphs[0]
            for x in graphs[1:]:
                mergedGraph = mergeTwoGraphs(mergedGraph, x)       
        return mergedGraph

In [54]:
merged_Graphs = mergeGraphs([dict_a2q_n,dict_c2a_n,dict_c2q_n])

x:  1
x:  3
x:  380
x:  4642
x:  2495
x:  2515
x:  292
x:  4213
x:  2353001
x:  4064
x:  4906
x:  3394
x:  811
x:  1597
x:  2915
x:  1615
x:  4285
x:  422
x:  4761
x:  445087
x:  797
x:  4926
x:  1891
x:  1573
x:  209
x:  2260
x:  4134
x:  4833
x:  3408
x:  1965
x:  1820
x:  4918
x:  5021
x:  4596
x:  4729
x:  1944
x:  2655
x:  3055
x:  3434
x:  1084
x:  1968
x:  2653
x:  227
x:  1996
x:  5076
x:  3848
x:  4489
x:  909
x:  2030
x:  1879
x:  2443
x:  369
x:  364
x:  146637
x:  1694
x:  1666
x:  2783
x:  720
x:  832
x:  46
x:  3641
x:  4433
x:  1127460
x:  4021
x:  959
x:  3955
x:  1008
x:  1908
x:  5083
x:  1709
x:  954
x:  999
x:  2961
x:  3377
x:  1920
x:  5056
x:  1242
x:  2385
x:  1412
x:  3043
x:  157
x:  2975
x:  383
x:  1790
x:  2194
x:  2361
x:  4725
x:  974
x:  3655
x:  2992
x:  1612
x:  1554
x:  730
x:  5058
x:  567
x:  3381
x:  1815
x:  3619
x:  770
x:  3474
x:  5119
x:  327
x:  3415
x:  672
x:  572
x:  2536
x:  4544
x:  5190
x:  1220
x:  4264
x:  35
x:  1385358
x:  2600
x:  

x:  16480
x:  1450
x:  12643
x:  2076
x:  14177
x:  6329
x:  7436
x:  11619
x:  3615
x:  12518
x:  14382
x:  10034
x:  142
x:  13894
x:  6770
x:  13824
x:  14238
x:  2135219
x:  11130
x:  12850
x:  16542
x:  12902
x:  16773
x:  14570
x:  16562
x:  13800
x:  6482
x:  16679
x:  15809
x:  15346
x:  14009
x:  9504
x:  12905
x:  8220
x:  12682
x:  13597
x:  10120
x:  8643
x:  12336
x:  8399
x:  1227001
x:  3350
x:  10906
x:  12711
x:  4281
x:  16911
x:  14351
x:  11388
x:  15534
x:  9974
x:  2214
x:  16948
x:  8114
x:  8787
x:  11538
x:  6014
x:  2994
x:  16398
x:  11544
x:  10890
x:  11344
x:  15486
x:  16980
x:  13276
x:  12842
x:  8256
x:  13005
x:  6852
x:  16050
x:  8995
x:  15992
x:  178
x:  2785
x:  1153
x:  1440933
x:  16703
x:  4919
x:  14484
x:  16977
x:  14057
x:  14028
x:  17076
x:  4089
x:  7611
x:  17092
x:  6223
x:  726
x:  9660
x:  1463
x:  17171
x:  6212
x:  396527
x:  16981
x:  16339
x:  16952
x:  9401
x:  14893
x:  373112
x:  14985
x:  17026
x:  17023
x:  11972
x:  5975
x

x:  10703
x:  10680
x:  2459
x:  2676
x:  20805
x:  19799
x:  93
x:  9634
x:  14167
x:  410
x:  4919
x:  16076
x:  9940
x:  11927
x:  11421
x:  9493
x:  11815
x:  13220
x:  21699
x:  19863
x:  15627
x:  11347
x:  12267
x:  15534
x:  4066
x:  3379
x:  1968
x:  8562
x:  13139
x:  5266
x:  17160
x:  4857
x:  1354
x:  21318
x:  13913
x:  7671
x:  15152
x:  7724
x:  326
x:  5379
x:  16314
x:  4843
x:  12765
x:  13930
x:  2766176
x:  3381
x:  615
x:  4642
x:  16942
x:  4853
x:  2648
x:  7154
x:  10862
x:  10026
x:  2477
x:  20654
x:  11374
x:  61
x:  3182
x:  20796
x:  7837
x:  15369
x:  6739
x:  2309
x:  21005
x:  2660
x:  3305
x:  15459
x:  10786
x:  446104
x:  11834
x:  7512
x:  12829
x:  18771
x:  6568
x:  12091
x:  507
x:  14532
x:  5946
x:  3743
x:  9632
x:  7595
x:  17028
x:  18821
x:  22656
x:  13531
x:  7644
x:  18037
x:  13253
x:  8718
x:  5599
x:  21640
x:  12579
x:  22468
x:  7122
x:  15541
x:  22748
x:  1811
x:  22445
x:  5696608
x:  9647
x:  9361
x:  9304
x:  8999
x:  17086
x: 

In [16]:
#merged_Graphs

In [ ]:
'''
dict_merged={}
dict_merged.update(dict_a2q_n)
for i in dict_c2a_n.keys():
    if i not in dict_merged.keys():
        dict_merged[i]={'c2vs_answer':c2a_n[0][1],'time_c2a':c2a_n[0][2],'weight':0}
for j in dict_c2q_n.keys():
    if j not in dict_merged.keys(): 
        dict_merged[j]={'c2vs_question':c2q_n[0][1],'time_c2q':c2q_n[0][2],'weight':0}
'''

In [ ]:
'''
for key in dict_merged.keys():
    if len(dict_merged[key])>3:
        print(i)
'''

#### Functionalities are defined as methods within a class.

In [2]:
import sys
from heapq import heappush, heappop
from itertools import count
import networkx as nx
# MYGRAPH = {'A': {'B': 2, 'C': 3}, 'B': {'A': 2, 'C': 5}, 'C':{'A':3, 'B':5, 'D': 10}}

MYGRAPH = {'A':{'B':5, 'C':3, 'D':4}, 'B':{'A':5, 'D':1}, 'C':{'A':3,'E':5}, 'D':{'A':4, 'B':1, 'E':2}}

# MYGRAPH = nx.erdos_renyi_graph(50,0.5)
# MYGRAPH is feed into the class in static way
# MYGRAPH is the merged graph and we need a function that returns the merged graph between two time intervals
class Graphs:
    def __init__(self, i):
        # <i> should be used in visualization phase.
        # We call each functionality (method) within the class according to
        # the given <i> by the user.
        self.i = i
        self.results = []
        self.init_graph = MYGRAPH
        self.nx_graph = self.construct_nx_graph()

    # First functionality
    def get_features(self, inp_graph):

        # Checking if the graph is undirected or not by looking for
        # similar edges between pairs of vertices.
        def is_directed():
            for u, neigh in inp_graph.items():
                for n in neigh.items():
                    if n[0] in inp_graph:
                        if n[0] != u and not ([(node, w) for node, w in inp_graph[n[0]].items() if (node == u and w == n[1])]):
                            # print("The graph is directed")
                            return 1
            # print("The graph is undirected.")
            return 0

        # Number of users is equal to all the unique values both in keys
        # and values of the given graph in the format of a dictionary.
        def num_of_users():
            _users = []
            for k, v in inp_graph.items():
                _users.append(k)
                for i in [*v]:
                    _users.append(i)
            return len(set(_users))

        # Counting all the values of the sub-dictionaries. If the graph is undirected
        # then we divide the result by two.
        def num_ans():
            _counter = 0
            for i in inp_graph.values():
                _counter += len(i)
            if self.results[0] == 0:
                return _counter/2
            else:
                return _counter

        # Number of edges / number of vertices
        def avg_links():
            return self.results[2]/self.results[1]

        # Two different scenarios based on whether the graph is directed
        # or not
        def density_degree():
            if self.results[0] == 1:
                _density = self.results[2] / (self.results[1]*(self.results[1] - 1))
            else:
                _density = 2 * self.results[2] / (self.results[1] * (self.results[1] - 1))
            return _density

        # if density < 0.5 then it is sparse; otherwise it is considered dense.
        def sparsity():
            if self.results[4] <= 0.5:
                # print("Sparse!")
                return 0
            else:
                # print("Dense!")
                return 1

        self.results.append(is_directed())
        self.results.append(num_of_users())
        self.results.append(num_ans())
        self.results.append(avg_links())
        self.results.append(density_degree())
        self.results.append(sparsity())

        return self.results

    # Making the graph dictionary symmetrical
    def construct_nx_graph(self):

        for k, v in self.init_graph.items():
            for index in v:
                v[index] = {'weight': v[index]}
        temp_graph = nx.DiGraph(self.init_graph)
        temp_graph.add_nodes_from(self.init_graph.keys())

        return temp_graph

    # Now we have the base for functionality #2
    def best_user(self, t_node):   #def best_user(self, t_node, init_time, end_time)
        # The section about graph merging needs to be reconsidered
        # We need to define a function

        # Extracting the desired graph between init_time and end_time here:
        ######################  Getting Output Graph  #####################
        # trimmed_graph = FUNCTION(init_time, end_time)
        # then replace self.nx_graph with trimmed_graph

        def betweeness():

            btwns = dict.fromkeys(self.nx_graph, 0.0)
            # s is the source
            S = []      # The shortest path
            P = {}      # The previous node in the shortest path

            # Initializing the previous path of each node as null
            for v in self.nx_graph:
                P[v] = []
            # Having a pure list of nodes in a dict with value of 0 of each unseen node and 1 for each seen one
            temp_dict = dict.fromkeys(self.nx_graph, 0.0)
            D = {}
            temp_dict[t_node] = 1.0      # Making the source node as seen
            Q = []  # out heap
            temp_push = heappush
            temp_pop = heappop
            seen = {t_node: 0}
            c = count()
            temp_push(Q, (0, next(c), t_node, t_node))
            while Q:
                (distance, _, pred, v) = temp_pop(Q)
                if v in D:
                    continue  # already searched this node.
                temp_dict[v] += temp_dict[pred]  # Counting the number of paths
                S.append(v)
                D[v] = distance
                for w, edges in self.nx_graph[v].items():
                    vw_dist = distance + edges.get('weight', 1)
                    if w not in D and (w not in seen or vw_dist < seen[w]):
                        seen[w] = vw_dist
                        temp_push(Q, (vw_dist, next(c), v, w))
                        temp_dict[w] = 0.0
                        P[w] = [v]
                    elif vw_dist == seen[w]:  # handle equal paths
                        temp_dict[w] += temp_dict[v]
                        P[w].append(v)

            # Then we accumulate the results of the Dijkstra without having any target (sink) node
            d_t = dict.fromkeys(S, 0)
            while S:
                t1 = S.pop()
                _coeffs = (1 + d_t[t1]) / temp_dict[t1]
                for node in P[t1]:
                    d_t[node] += temp_dict[node] * _coeffs
                if t1 != t_node:
                    btwns[t1] += d_t[t1]

            return btwns
        
        betweeness()
        
# Testing with both directed and undirected (dense) graphs
# uncomment for directed graph
# my_dict = {'A': {'B': 2, 'C': 3}, 'B': {'A': 4, 'C': 5, 'D':10}}
# uncomment for undirected graph
# my_dict = {'A': {'B': 2, 'C': 3}, 'B': {'A': 2, 'C': 5}, 'C':{'A':3, 'B':5}}

# i = 1 indicates functionality 1
# graph_features = Graphs(1)
# report = graph_features.get_features(my_dict)
# print(report)

g = Graphs(1)
print(g.best_user('C'))

None


In [3]:
my_dict = {'A': {'B':{'weight': 2}, 'C':{'weight': 3}}, 'B': {'A':{'weight': 4}, 'C':{'weight': 5}, 'D':{'weight': 10}}}

In [ ]:
r=nx.Graph(my_dict)

In [ ]:
r.nodes()

# FUNCTIONALITY 3

In [25]:
def dijkstra(g,start,end):
    s=start
    route=[]
    #remaining_seq= [x for x in seq if (x not in route)]
    dist={}
    finalized={}
    for i in g.keys(): #initialize visited as false for all nodes
        dist[i]= np.inf
        finalized[i]=False
    dist[s]=0
    finalized[s]=True
    route.append(s)
    #remaining_seq.remove(s)
    while s != end:
        adj=[]
        for u in list(g[s].keys())  : #for every neighbor of s
            if u in g:
                if finalized[u]==False: #if it's not finalized
                    adj.append(u) #add to adjacent nodes which aren't finalized, then update distance
                    x=dist[s]+g[s][u]['weight']
                    if dist[u]>x:
                        dist[u]=x
        for k in adj:
            if k==end: # if the end node is adjacent
                route.append(k) #add end to route, then end
                return (route)
        if len(adj)==0: #if we reach a leaf
            route.remove(s) #remove it
            s=route[-1] #return to previous node
            route.remove(s) #remove duplicate of previous node
        else:
            m=[k for k, v in dist.items() if v ==min(dist[a] for a in adj)] #get node/nodes that has/have minimum distance
            m=[k for k in m if (finalized[k]==False and k in adj)] #remove the ones not adjacent or finalized
            s=m[-1] #take last one
        finalized[s]=True
        route.append(s)
        adj=[]
    return(route)

In [74]:
def in_same_component(x,y,g): # to check if nodes x and y are in the same connected component in graph g
    neighbors=[x]
    for i in neighbors:
        if i in merged_Graphs:
            for u in list(merged_Graphs[i].keys()):
                if u not in neighbors:
                    if u in g:
                        neighbors.append(u)
    if y in neighbors:
        return True
    return False
    

In [70]:
def shortest_ordered_route(initTime, endTime, seq, start,end): #take p_1 (or p_j) and p_n as input
    g=nx.Graph(createGraph(initTime, endTime, merged_Graphs))
    route=[]
    route.extend(dijkstra(g,start,seq[0]))
    for i in range(0,len(seq)-1):
        route.extend(dijkstra(g,seq[i],seq[i+1])[1:])
    route.extend(dijkstra(g,seq[-1],end)[1:])
    return(route)

In [71]:
def graph_fun3(a2q,c2a,c2q,initTime,endTime):
    a2q_fun3=[i for i in a2q if initTime <= i[2] <=endTime]
    c2a_fun3=[i for i in c2a if initTime <= i[2] <=endTime]
    c2q_fun3=[i for i in c2q if initTime <= i[2] <=endTime]
    g=mergeGraphs([createGraph('-', '-', a2q_fun3),createGraph('-', '-', c2a_fun3),createGraph('-', '-', c2q_fun3)])
    return g

In [72]:
def shortest_ordered_route(initTime,endTime, seq, start, end):
    g=graph_fun3(a2q,c2a,c2q,initTime,endTime)
    if in_same_component(start,end,g)==True:
        route=[]
        route.extend(dijkstra(g,start,seq[0]))
        for i in range(0,len(seq)-1):
            route.extend(dijkstra(g,seq[i],seq[i+1])[1:])
        route.extend(dijkstra(g,seq[-1],end)[1:])
        return(route)
    return(print("Not possible"))

In [76]:
shortest_ordered_route('2008-08-01', '2008-09-09', [404,1188,40],9,1982)

IndexError: list index out of range

In [3]:
dt.utcfromtimestamp(int(c2q[len(c2q)-1][2])).strftime('%Y-%m-%d %H-%M-%S')

'1970-01-01 00-00-07'

In [5]:
dt.utcfromtimestamp(int(c2q[0][2])).strftime('%Y-%m-%d %H-%M-%S')

'1970-01-01 00-00-05'

## FUNCTIONALITY 4

In [ ]:
def nodesConnected(u,v, g): #return true if you can reach de node 'v' from node 'u' in the graph 'g'
    toVisitNodes = set([u])
    visitedNodes = set()
    
    while len(toVisitNodes) > 0:
        node = toVisitNodes.pop()
        visitedNodes.add(node)
        #print("-------------------")
        #print("We are in node: ", node)
        #print("VisitedNodes: ", visitedNodes)
        neighbors = []
        if node in g:
            neighbors = list(g[node].keys())
        for x in neighbors:
        #    print("we are checking neighbore: ", x)
            if x in g: #if neighbor it's on the graph, we continue. If not, we can't reach it
                if x == v: #we found the node 'v', so we stop and return True
                    return True
                elif x not in visitedNodes: #if it's not our objective node, and we haven't visited yet, we add it to the nodes to visit.
                    toVisitNodes.add(x)
        #print("-------------------")
    return False

def recursiveDFS(node, visitedNodes, g):
    visitedNodes.add(node)
    print(node, end=' ')
    
    neighbors = []
    if node in g:
        neighbors = list(g[node].keys()) 
    
    for x in neighbors:
        if x not in visitedNodes:
            recursiveDFS(x, visitedNodes, g)
    

def DFS(node, graph):
    visitedNodes = set()
    recursiveDFS(node, visitedNodes, graph)

def fordFulkerson():
    return True

In [ ]:
graph_a2q_n = createGraph('2008-08-01', '2008-09-09', a2q_n)
graph_c2a_n = createGraph('2008-08-01', '2008-09-09', c2a_n)
graph_c2q_n = createGraph('2008-08-01', '2008-09-09', c2q_n)

merged_Graphs_func4 = mergeGraphs([graph_a2q_n,graph_c2a_n,graph_c2q_n])


In [ ]:
DFS(3, my_dict)

user1 = 9
user2 = 8

#As we are working with a directed graph, we check if from user 1 we can reach user 2 and vice versa. So if we cant find a path
#that connect users in both directions, we have finished with the objective of disconnect the users.
if (not nodesConnected(user1,user2,merged_Graphs_func4)) and (not nodesConnected(user2,user1,merged_Graphs_func4)): 
    print("The two users are not connected.")
else:
    
#print(nodesConnected(user1,user2,merged_Graphs_func4))
#print(nodesConnected(user2,user1,merged_Graphs_func4))

In [ ]:
merged_Graphs_func4

In [ ]:
my_dict = {1: {2:{'weight': 2}, 3:{'weight': 3}}, 2: {3:{'weight': 4}}, 3: { 4: {'weigth': 2}}, 4: {2: {'weight':2}} }

In [ ]:
nodesConnected(1,4,my_dict)

In [ ]:
list(dict_a2q_n[9].keys())

In [ ]:
dict_a2q_n